Scheduler Agent

### Importing Required Libraries

In [994]:
import os
from openai import OpenAI
import json
from vllm import SamplingParams
import requests
from typing import List, Dict, Any

## Deploying the LLM using vLLM

Start deploying the LLM (/deepseek-ai/deepseek-llm-7b-chat) using vLLM in the Jupyter notebook:

### Start the vLLM server 

Open a new tab in this Jypyter server, click on the terminal icon to open a new terminal, then copy the following command to launch the vLLM server:

```bash
HIP_VISIBLE_DEVICES=0 vllm serve /home/user/Models/deepseek-ai/deepseek-llm-7b-chat \
        --gpu-memory-utilization 0.9 \
        --swap-space 16 \
        --disable-log-requests \
        --dtype float16 \
        --max-model-len 2048 \
        --tensor-parallel-size 1 \
        --host 0.0.0.0 \
        --port 3000 \
        --num-scheduler-steps 10 \
        --max-num-seqs 128 \
        --max-num-batched-tokens 2048 \
        --max-model-len 2048 \
        --distributed-executor-backend "mp"
```

In [995]:
url = "http://localhost:4000/v1/chat/completions"
headers = {"Content-Type": "application/json"}
MODEL_PATH = '/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct'

In [996]:
from datetime import datetime
import pytz

ist = pytz.timezone('Asia/Kolkata')
current_date_time = datetime.now(ist)
current_day = current_date_time.strftime('%A')
current_year = current_date_time.year

2025-07-20 13:07:42.601522+05:30
Sunday
2025


In [997]:
attendees = ['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']

In [998]:
system_prompt = f"""
Yor are an Agent that helps in scheduling meetings.
You are given a request which will include all meeting attendees and the potential date and time of the meeting.
Your job is to fetch the calendar of all attendees, and find a free slot that works for all of them.

You have access to the following tools:
1. find_meeting_slots(): To find free slots, given the time period, duration of the meeting and the emails of all the users.
This tool will either output 1 free slot if found, otherwise it will output an empty result.

All events are in IST timezone.
Current datetime is: {current_date_time}
Current day is: {current_day}
Current year is: {current_year}

One other task that you have is to prioritise the current meeting as either low(0) or high(1) priority.
This can be done based on the meeting description.
For example: any client call or major bug is high priority whereas one to one meetings, syncs or regular meetings are lower priority.

Your first step is to call a tool to find_meeting_slots() for all users in a defined time range.
Once you have found a free slot, you must not call the tool again. You should just return the free slot in the output.
For calling a tool, you should return a json output with the following keys:
1. 'tool_name': string
2. 'tool_arguments': dict

If you are trying to get the free slots for the first time, call the first tool: find_meeting_slots()
YOU CAN CALL THIS TOOL ONLY ONCE. DO NOT CALL IT AGAIN IN ANY SCENARIO

Sample arguments for tool find_meeting_slots():
1. user_emails: List of request user along with all attendees
2. start_time: datetime format
3. end_time: datetime format
4. duration_minutes: integer
5. meeting_priority: integer 

If you have found a free slot:
DO NOT MAKE THE TOOL CALL AGAIN
you must return the free slot as your output in the below format:
1. 'free_slot_start_timestamp': datetime
2. 'free_slot_end_timestamp': datetime
3. 'duration': int (in mins)
4. 'attendee_schedule': dict[str, list] - List of calendar events for each user.

How to determine the tool arguments?
1. Check the meeting description from the user request and try to decode.
2. Keep start_time and end_time as broad as possible within the given constraints.
3. You can consider a day to start from mightnight(00:00) and end at midnight(00:00) of the next day.
For example to schedule a meeting on 30th July, the start_time will be 30th July 00:00 and end time will be 31th July 00:00.
4. But if you are specified a certain time within the day, make sure you ensure those constraints in the start time and end time.
5. If you are given a week to schedule, you can keep a gap of 7 days between start time and end time.
and so on.

VERY IMPORTANT INSTRUCTIONS:
1. OUTPUT MUST ONLY CONTAIN A JSON. FREE TEXT IS NOT ALLOWED. JSON! JSON! JSON! ONLYYY
2. IF LAST MESSAGE HAS A FREE SLOT, DO NOT CALL THE TOOL AGAIN. JUST RETURN THE FREE SLOT IN OUTPUT AS PER DESCRIPTION.
3. Stricty follow the instructions. Strictly return the above json only. 
4. Do not add any other instrctions or information. 

Attendees: {','.join(attendees)}
"""

In [999]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "find_meeting_slots",
            "description": "Find free meeting slots for multiple users within a specified time range",
            "parameters": {
                "type": "object",
                "properties": {
                    "users": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of user email addresses to fetch calendars for"
                    },
                    "start_time": {
                        "type": "string",
                        "description": "Start time in ISO format (e.g., '2025-07-18T00:00:00+05:30')"
                    },
                    "end_time": {
                        "type": "string",
                        "description": "End time in ISO format (e.g., '2025-07-18T23:59:59+05:30')"
                    },
                    "duration_minutes": {
                        "type": "integer",
                        "description": "Duration of meeting slots in minutes"
                    },
                    "meeting_priority": {
                        "type": "boolean",
                        "description": "Based on the meeting description, prioritise it as either high(1) or low(0) priority."
                    }
                },
                "required": ["users", "start_time", "end_time", "duration_minutes", "meeting_priority"]
            }
        }
    }
]

In [1000]:
url = "http://localhost:4000/v1/chat/completions"
headers = {"Content-Type": "application/json"}
model_path = "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct"

class SCHEDULER_AGENT:
    def __init__(self):
        pass

    def _execute_tool(self, function_name: str, arguments: Dict[str, Any]) -> str:
        """Execute the requested tool function"""
        try:
            start_time = arguments.get('start_time')
            start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
            ist = pytz.timezone('Asia/Kolkata')
            start_time = ist.localize(start_time)

            end_time = arguments.get('end_time')
            end_time = datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
            ist = pytz.timezone('Asia/Kolkata')
            end_time = ist.localize(end_time)

            # if function_name == "fetch_calendar_events":
            #     tool_output = await fetch_calendar_events(
            #         arguments.get('users'),
            #         start_time,
            #         end_time
            #     )
            #     print(f'\n\n\n {MEETINGS_DB} \n\n\n')
            #     return tool_output
            if function_name == "find_meeting_slots":
                tool_output = find_meeting_slots(
                    arguments.get('user_emails'),
                    start_time,
                    end_time,
                    arguments.get('duration_minutes'),
                    arguments.get('meeting_priority')
                )
                return json.dumps(tool_output) if tool_output else {"message": "No free slots found"}
        except Exception as e:
            print(f'Exception in executing tool: {e}')


    def run_llm(self, user_request):
        messages = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_request
            }
        ]
        for iteration in range(3):
            agent_data = {
                "model": model_path,
                "messages": messages,
                "stream": False,
                "tools": tools,
                "tool_choice": "none",
                "max_tokens": 1024,
                "temperature": 0.5
            }
            try:
                response = requests.post(url, headers=headers, json=agent_data)
                result = response.json()

                message = json.loads(result.get('choices', [{}])[0].get('message', {}).get('content', {}))
                if not message.get('tool_name'):
                    return message

                tool_name = message.get('tool_name')
                arguments = message.get('tool_arguments')
                
                tool_result = self._execute_tool(tool_name, arguments)

                return tool_result

                # Add tool result to conversation
                messages.append({
                    "role": "system",
                    "content": tool_result,
                })
            except Exception as e:
                return f"Unexpected error occured in executing llm call: {e}"
        return "Maximum iterations reached. Please try a simpler request."

In [1001]:
scheduler_agent = SCHEDULER_AGENT()

In [1002]:
%run Calendar_Event_Extraction.ipynb
%run findFreeSlot.ipynb

In [1003]:
# await scheduler_agent.run_llm('Hi team, schedule a high priority meeting today at 1pm for 9 hours')